# Groupby 2: Aggregation and expressions
By the end of this lecture you will be able to:
- aggregate groups with `agg`
- sort the output of an aggregation
- group on a transformed column
- filter an aggregation

In [ ]:
import polars as pl
import plotly.express as px

In [ ]:
csv_file = '../data/titanic.csv'

In [ ]:
df = pl.read_csv(csv_file)
df.head(3)

## Aggregate a single column

We aggregate by calling `agg` after `group_by` and passing one or more expressions. In this example we get the mean age of passengers in each class

In [ ]:
(
    df
    .group_by('Pclass')
    .agg(
        pl.col('Age').mean()
    )
)

We must use expressions inside `agg`, we cannot use column name strings.

To sort the output by group key or aggregation column we must use `sort`. In this example we order by the group key

In [ ]:
(
    df
    .group_by('Pclass')
    .agg(
        pl.col('Age').mean()
    )
    .sort('Pclass')
)

Note that if we don't specify an aggregating expression - such as taking the average in each group - then we get a `pl.List` column with all the values for each group

In [ ]:
(
    df
    .group_by('Pclass')
    .agg(
        pl.col('Age')
    )
)

## Groupby on an expression
We can use expressions inside the `group_by` method to transform the column before grouping.

We want to see if passengers with longer names paid a higher fare on average. We do this by:
- transforming the `Name` column to get the number of characters
- getting the average `Fare` per length of `Name`
- sorting by the average `Fare`


In [ ]:
(
    pl.read_csv(csv_file)
    .group_by(
        pl.col("Name").str.len_chars()
    )
    .agg(
        pl.col("Fare").mean()
    )
    .sort("Fare",descending=True)   
)

The output is a little tricky to interpret. We can easily visualise the results by `piping` the output to a `px.scatter` plot

In [ ]:
(
    pl.read_csv(csv_file)
    .group_by(
        pl.col("Name").str.len_chars()
    )
    .agg(
        pl.col("Fare").mean()
    )
    .sort("Fare",descending=True)
    .pipe(
        px.scatter,
        x="Name",
        y="Fare",
        width=500
    )
)

These results suggest that there is a loose positive relationship between length of name and price of ticket

## Apply a filter on an aggregation

We may want to filter the results after doing the aggregation so that only some of the aggregates appear in the output.

> In SQL this is done using a `HAVING` statement

We do this in Polars with an additonal `filter` after calling `agg`.

In this example we get the average fare by passenger class but only if the average fare by class is greater than £20

In [ ]:
(
    pl.read_csv(csv_file)
    .group_by("Pclass")
    .agg(
        pl.col("Fare").mean()
    )
    .filter(
        pl.col("Fare") > 20
    )
    .sort("Fare")
)

## Exercises

In the exercises you will develop your understanding of
- doing `group_by` on a column
- using an expression in a `group_by`
- applying a `filter` on an aggregation

### Exercise 1
Get the average fare by `Age` 

In [ ]:
(
    pl.read_csv(csv_file)
    .<blank>
)

Round the `Age` column to the nearest year before doing the group_by. Sort the output by age.

Continuing from the previous cell, output the rows where the average fare is greater than 30

### Exercise 2

Group by the `Pclass` column. Count the number of passengers in each group in a column called `count`

In [ ]:
(
    pl.read_csv(csv_file)
    <blank>
)

Add a column called `percent` with the percentage of the total passengers in each group. Do this by: 
- diving the values in `count` by the sum of the values in `count`
- multiplying by 100

Create a bar chart of the `percent` column by piping the output to `px.bar` with the title `% per class"`

## Solutions

### Solution to Exercise 1
Get the average fare by `Age` 

In [ ]:
(
    pl.read_csv(csv_file)
    .group_by('Age')
    .agg(
        pl.col("Fare").mean()
    ).head(2)
)

Round the `Age` column to the nearest year before doing the group_by. Sort the output by age.

In [ ]:
(
    pl.read_csv(csv_file)
    .group_by(
        pl.col('Age').round(0)
    )
    .agg(
        pl.col("Fare").mean()
    )
    .sort('Age')
)

Continuing from the previous cell, output the rows where the average fare is greater than 30

In [ ]:
(
    pl.read_csv(csv_file)
    .group_by(
        pl.col('Age').round(0)
    )
    .agg(
        pl.col("Fare").mean()
    )
    .sort("Age")
    .filter(pl.col("Fare") > 30)
    .head()
)

### Solution to Exercise 2

Group by the `Pclass` column. Count the number of passengers in each group in a column called `count`

In [ ]:
(
    pl.read_csv(csv_file)
    .group_by("Pclass")
    .agg(pl.col("Name").count().alias("count"))
)

Add a column called `percent` with the percentage of the total passengers in each group. Do this by: 
- diving the values in `count` by the sum of the values in `count`
- multiplying by 100

In [ ]:
(
    pl.read_csv(csv_file)
    .group_by("Pclass")
    .agg(pl.col("Name").count().alias("count"))
    .with_columns(
        (100 * (pl.col("count") / pl.col("count").sum())).alias("percent")
    )
)

Create a bar chart of the `percent` column by piping the output to `px.bar` with the title `% per class"`

In [ ]:
(
    pl.read_csv(csv_file)
    .group_by("Pclass")
    .agg(pl.col("Name").count().alias("count"))
    .with_columns(
        (100 * (pl.col("count") / pl.col("count").sum())).alias("percent")
    )
    .pipe(
        px.bar,
        x="Pclass",
        y="percent",
        title="% per class"
    )
)